# Basic examples from trimAl

This example shows how to run the basic methods shown in the trimAl manual, but using the `pytrimal` API.

In [ ]:
import pytrimal
pytrimal.__version__

For this example, we will use an alignment used to generate HMMs for Pfam (namely [PF12574](https://www.ebi.ac.uk/interpro/entry/pfam/PF12574/), a family of Rickettsia surface antigen). We use `Alignment.load` to load the alignment from a filename; note that `os.PathLike` objects are supported as well.

In [ ]:
import pathlib
ali = pytrimal.Alignment.load(pathlib.Path("data").joinpath("PF12574.full.afa"))

To easily compare the alignments, let's use [rich-msa](https://pypi.org/project/rich-msa), a Python package for displaying multiple sequence alignments with [rich](https://github.com/Textualize/rich).

In [ ]:
import rich.console
import rich.panel
from rich_msa import RichAlignment

def show_alignment(alignment):
    console = rich.console.Console(width=len(alignment.sequences[0])+40)
    widget = RichAlignment(names=[n.decode() for n in alignment.names], sequences=alignment.sequences, max_name_width=30)
    panel = rich.panel.Panel(widget, title_align="left", title="({} residues, {} sequences)".format(len(alignment.sequences[0]), len(alignment.sequences)))
    console.print(panel)

Let's see how the original alignment looks before trimming:

In [ ]:
show_alignment(ali)

## Example 1

Remove all positions in the alignment with gaps in 10% or more of the sequences, unless this leaves less than 60% of original alignment. In such case, print the 60% best (with less gaps) positions. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -gt 0.9 -cons 60
```

In [ ]:
trimmer = pytrimal.ManualTrimmer(gap_threshold=0.9, conservation_percentage=60)
trimmed = trimmer.trim(ali)
show_alignment(trimmed)

## Example 2

Same as *Example 1*, but  the gap score is averaged over a window starting 3 positions before and ending 3 positions after each column.

In [ ]:
trimmer = pytrimal.ManualTrimmer(gap_threshold=0.9, conservation_percentage=60, window=3)
trimmed = trimmer.trim(ali)
show_alignment(trimmed)

## Example 3

Use an automatic method to decide optimal thresholds, based on the gap scores the input alignment. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -gappyout
```

In [ ]:
trimmer = pytrimal.AutomaticTrimmer(method="gappyout")
trimmed = trimmer.trim(ali)
show_alignment(trimmed)

## Example 4

Use automatic methods to decide optimal thresholds, based on the combination of gap and similarity scores. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -strictplus
```

In [ ]:
trimmer = pytrimal.AutomaticTrimmer(method="strictplus")
trimmed = trimmer.trim(ali)
show_alignment(trimmed)

## Example 5

Use an heuristic to decide the optimal method for trimming the alignment. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -automated1
```

In [ ]:
trimmer = pytrimal.AutomaticTrimmer(method="automated1")
trimmed = trimmer.trim(ali)
show_alignment(trimmed)

## Example 6

Use residues and sequences overlap thresholds to delete some sequences from the alignment. See the [trimAl User Guide](http://trimal.cgenomics.org) for details.

```
$ trimal -in data/example.001.AA.clw -resoverlap 0.6 -seqoverlap 75
```

In [ ]:
trimmer = pytrimal.OverlapTrimmer(residue_overlap=0.6, sequence_overlap=75)
trimmed = trimmer.trim(ali)
show_alignment(trimmed)

## Example 10

Select the 5 most representative sequences from the alignment.

```
$ trimal -in data/example.001.AA.clw -clusters 5
```

In [ ]:
trimmer = pytrimal.RepresentativeTrimmer(clusters=5)
trimmed = trimmer.trim(ali)
show_alignment(trimmed)